In [1]:
import shutil

source_file_path = '.\..\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [2]:
import django
import os
import sys
project_path = '../../'  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()

import re
from asgiref.sync import sync_to_async

from PostAnalyzer.models import (
    Market,
    Symbol,
    PositionSide,
    MarginMode,
)
import warnings

from Shared.helpers import returnSearchValue

from dotenv import dotenv_values

config = dotenv_values(".env")

api_id = config["api_id"]
api_hash = config["api_hash"]

username = config["username"]
error_msg = []


allPredicts = []

historic_json_path = "./../historic-json"


# Disable all warnings
warnings.filterwarnings("ignore")

## is Predict Msg?

In [3]:
def isPredictMsg(msg):
    patterns = [
        r"📌 #(.+)",
        r"Entry:(.+)",
        r"Leverage:(.+)",
        r"Stop : (.+)",
        r"TP:(.+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)


## Symbol

In [4]:
async def findSymbol(msg):
    symbol = re.search(r"📌 #(.+)?USDT", msg, re.IGNORECASE)
    
    try:
        return await sync_to_async(Symbol.objects.get)(asset=returnSearchValue(symbol).upper())
    except:
        return None

## Market

In [5]:
async def findMarket(msg):

    market = None
    if "Futures Call".lower() in msg.lower():
        market = "FUTURES"
    elif "Spot".lower() in msg.lower():
        market = "SPOT"
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=market.upper())
        return market_value
    except:
        return None


## Position

In [6]:
async def findPosition(msg):
    if "short" in msg.lower():
        pos = "SHORT"
    elif "long" in msg.lower():
        pos = "LONG"

    try:
        market_value = await sync_to_async(PositionSide.objects.get)(name=pos)
        return market_value
    except:
        return None

## Leverage and Margin Mode

In [7]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_value = int(leverage_match.group(1).lower().replace("x","")) 
        # leverage_value = int(leverage_match.group(1)[2].lower().replace("x",""))   
    else:
        leverage_value = None

    leverage_type = await sync_to_async(MarginMode.objects.get)(name="ISOLATED")   
    
    return leverage_type, leverage_value

## StopLoss

In [8]:
def findStopLoss(msg):
   msg = msg.replace(",","")
   msg = msg.replace("‌","")
 
   return returnSearchValue(re.search(r"Stop\s*:\s*(?:کلوز)?\s*([\d.]+)", msg))

## Entry Targets

In [9]:
def findEntryTargets(msg):
    entry_targets_match = re.search(r"Entry:(.+?)\n\n", msg, re.DOTALL)
    entry_values = (
        re.findall(r"\d+(?:\.\d+)?", entry_targets_match.group(1))
        if entry_targets_match
        else None
    )
    return [float(x.strip()) for x in entry_values]

## Take Profits

In [10]:
def findTakeProfits(msg):
    take_profit_targets_match = re.search(r"TP:(.+?)\n\n", msg, re.DOTALL)
    profit_values = (
        re.findall(r"\d+(?:\.\d+)?", take_profit_targets_match.group(1))
        if take_profit_targets_match
        else None
    )
    return [float(x.strip()) for x in profit_values]
        

## test samples

In [11]:

msg = "📣Futures Call Scalp 15min\n\n  💹LONG\n\n📌 #BTCUSDT\n\n💰 Entry: 57300-56000\n\n🏹 TP:57800-58200-58600-59000\n59500-60000-60500-61000\n\n🧨 Stop : 54980\n\n⚖️ Leverage:10X-20X\n\n✅ Capital %: 3-5\n\nمیتونه یه پولبک‌بزنه و‌نوسانشو بگیریم\n1403.04.15\n〰️〰️〰️〰️〰️〰️〰️"
msg1 = "📣 Futures Call Scalp\n\n  💹SHORT\n\n📌 #TRXUSDT\n\n💰 Entry: 0.12912-0.137\n\n🏹 TP:0.126-0.123-0.12-0.188-0.161-0.13\n0.109-0.105-0.103-0.1-0.095-0.089-0.08-0.07\n\n🧨 Stop : 0.146\n\n⚖️ Leverage: 3x-5X-8X\n\n✅ Capital %: 3-5\n\n\n1403.04.17\n〰️〰️〰️〰️〰️〰️〰️"
msg2 =  "📣 Futures Call Scalp\n\n  💹SHORT\n\n📌 #AAVEUSDT\n\n💰 Entry: 97.55-104\n\n🏹 TP:94.9-93.5-92-90.5-89.4-88.3-87-85-82-80-77-74-70-65\n\n🧨 Stop : 107.89\n\n⚖️ Leverage: 5X-8x-10X\n\n✅ Capital %: 3\n\nسیگنال قبل سود خوبی دادمجدد میشه وارد شد\nزیر این سطح سلریم\n\n1403.04.24\n〰️〰️〰️〰️〰️〰️〰️\n‌‌"
msg3 =  "📣 Futures Call Scalp \n\n  💹SHORT\n\n📌 #EGLDUSDT\n\n💰 Entry: 35.88-37.8\n\n🏹 TP:35-34.3-33.7-33.1-32.5-32-31.5-31\n-30.8-30.4-30-29-28.2-27.5-26\n\n🧨 Stop : 38.92\n\n⚖️ Leverage: 5X-10X\n\n✅ Capital %: 3\n\nخب هردو پله ورود فعال شد با میانگین 35.72 فعال شد تارگتهارو اپدیت میکنم دوستانیم که ندارن میتونن استفاده کنند.بالا زدنشم طبق مسیر بوده\nمادامی که بالای ترند تثبیت نده پایین میزنه\n\n1403.04.25\n〰️〰️〰️〰️〰️〰️〰️\n‌‌"
msg4 = "📣 Futures Call Scalp\n\n  💹SHORT\n\n📌 #Btcusdt\n\n💰 Entry: 63000-65300\n\n🏹 TP:62400-61900-61000-60000-59100-58000-56000-54000-52000-50000-48000-44000-40000\n\n🧨 Stop : 66250کلوز\n\n⚖️ Leverage: 5X-10X-15X\n\n✅ Capital %: 3\n\nموج۴ تحلیل قبل مثلث بود که‌تغییر ساختار داد\nحالا دیگه بنظرم  حتی میتونه سطح۵۲-۵۰ هم‌بشکنه و حمایت نشه \n\n1403.04.25\n〰️〰️〰️〰️〰️〰️〰️\n‌‌"
msg5 = "📣Futures Call\n\n  💹LONG\n\n📌 #SNXUSDT\n\n💰 Entry: 2.865-2.533\n\n🏹 TP:3-3.12-3.26-3.4-3.58-3.67-3.78-3.9-4.2-4.46-4.7-4.92-5.3-5.7-6-6.3-6.8\n\n🧨 Stop : کلوز‌2.37\n\n⚖️ Leverage: 2x-3x-5X\n\n✅ Capital %: 3-5\n\n\nسیگنال قبل سود خوبی داد بنظرم برگشتها پولبکه\n1403.03.03\n〰️〰️〰️〰️〰️〰️〰️"
msg6 =  "📣Futures Call\n\n  💹LONG\n\n📌 #THETAUSDT\n\n💰 Entry: 2.3-1.99\n\n🏹 TP:2.4-2.48-2.56-2.67-2.75-2.85-2.94-3\n3.1-3.23-3.4-3.7-4-4.5-5-5.4-5.8-6.5-7.2\n\n🧨 Stop : کلوز‌1.84\n\n⚖️ Leverage: 2x-3x-4X\n\n✅ Capital %: 3-5\n\nتتا دو‌حرکت بزرگشو‌گرفتیم از کف که‌لانگ دادیم \n365% رشد داشت و سقف سلمون 57% ریزش اسپات حالا بنظر موج 4 اصلاحی تموم شده و‌اماده رشد در قالب موج 5 باشه\n\n1403.03.19\n〰️〰️〰️〰️〰️〰️〰️"

msg = msg6

isPredict  = isPredictMsg(msg)
print(isPredict)

symbol = await findSymbol(msg)
print(symbol)

market = await findMarket(msg)
print(market)

pos = await findPosition(msg)
print(pos)

mode, lev = await findLeverage(msg)
print(mode, lev)

stop = findStopLoss(msg)
print(stop)

et = findEntryTargets(msg)
print(et)

tp = findTakeProfits(msg)
print(tp)






True
THETA-USDT
FUTURES
LONG
ISOLATED 2
1.84
[2.3, 1.99]
[2.4, 2.48, 2.56, 2.67, 2.75, 2.85, 2.94, 3.0, 3.1, 3.23, 3.4, 3.7, 4.0, 4.5, 5.0, 5.4, 5.8, 6.5, 7.2]


In [15]:
# # SHORT
# from bingx.api import BingxAPI
# from dotenv import dotenv_values

# config = dotenv_values(".env")


# API_KEY = config["API_KEY"]
# SECRET_KEY = config["SECRET_KEY"]

# bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")


# entry_price = [71260, 73400]
# stop_loss = 74800 
# take_profit = [69500, 68400 , 67500, 66300, 65000, 64000]
# symbol = "BTC-USDT" 
# time_interval = "1m"
# start_timestamp = 1716272400000

# # 86400000 == milliseconds in a day
# next_day = start_timestamp + 86400000

# reached_target = None
# stop_loss_reached = None
# take_profit_reached = None
# tps = []
# xxx = []
# i = 1

# tp_turn = 0
# tp = take_profit[tp_turn ]

# entry_turn = 0
# entry = entry_price[entry_turn]
# entry_reached = []

# stop_loss_reached = None
# keepOn = True
# limit = 1440
# milliSecInDay = 86400000

# while keepOn:
#     res = bingx.get_kline_data(symbol, time_interval ,start_timestamp ,next_day, limit)
#     xxx += list(reversed(res))

#     for row in reversed(res):
        
#         if not entry_reached and float(row['high']) >= float(entry):
#             entry_reached.append(row)

#             if entry_turn < len(entry_price)-1:
#                 entry_turn += 1
#                 entry = entry_price[entry_turn]
#             else:
#                 entry = float("inf")

#             continue

#         if entry_reached and float(row['high']) >= float(entry):
#             entry_reached.append(row)

#             if entry_turn < len(entry_price)-1:
#                 entry_turn += 1
#                 entry = entry_price[entry_turn]
#             else:
#                 entry = float("inf")

#             continue

    
#         if bool(entry_reached):
#             if not bool(stop_loss_reached) and float(row['high']) >= float(stop_loss):
                
#                 stop_loss_reached = row
#                 keepOn = False
#                 break

#             if float(row['low']) <= float(tp):
                
#                 tps.append(row)
#                 tp_turn += 1

#                 if (tp_turn) == len(take_profit):
#                     keepOn = False
#                     break
#                 tp = take_profit[tp_turn]
        

#     start_timestamp = next_day
#     next_day = start_timestamp + milliSecInDay
    
#     if len(res) < limit:
#         keepOn = False
#         break



# # print(take_profit_reached)
# print("stop_loss: ", stop_loss_reached)
# print("entry: " , entry_reached)
# print(tps)
# convertToJsonFile(xxx,"2x",".")


stop_loss:  None
entry:  [{'open': '71208.1', 'close': '71257.1', 'high': '71269.0', 'low': '71208.1', 'volume': '38.23', 'time': 1716272760000}]
[{'open': '69630.8', 'close': '69402.2', 'high': '69632.1', 'low': '69361.4', 'volume': '144.03', 'time': 1716319620000}, {'open': '68643.7', 'close': '68483.3', 'high': '68692.2', 'low': '68387.3', 'volume': '224.93', 'time': 1716471960000}, {'open': '67558.2', 'close': '67602.1', 'high': '67614.8', 'low': '67462.5', 'volume': '112.02', 'time': 1716487320000}, {'open': '66650.3', 'close': '66249.0', 'high': '66654.5', 'low': '66249.0', 'volume': '348.07', 'time': 1718120400000}, {'open': '65315.1', 'close': '64736.1', 'high': '65331.4', 'low': '64669.1', 'volume': '490.22', 'time': 1718674980000}, {'open': '64088.8', 'close': '63819.1', 'high': '64091.8', 'low': '63519.4', 'volume': '511.54', 'time': 1718959740000}]
